In [2]:
import pandas as pd
import numpy as np

import datetime as dt

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
data = pd.DataFrame([
            ['16-10-2021', '23:56:10', 'A', 'a1', '0001', np.random.randint(100)], 
            ['16-10-2021', '23:58:30', 'A', 'a1', '0002', np.random.randint(100)],
            ['16-10-2021', '23:58:35', 'B', 'b1', '0003', np.random.randint(100)],
            ['16-10-2021', '23:58:56', 'B', 'b2', '0004', np.random.randint(100)],
            ['16-10-2021', '23:59:21', 'C', 'c1', '0005', np.random.randint(100)],
            ['16-10-2021', '23:59:42', 'C', 'c1', '0006', np.random.randint(100)],

            ['17-10-2021', '00:03:05', 'A', 'a1', '0007', np.random.randint(100)],
            ['17-10-2021', '00:03:25', 'A', 'a2', '0008', np.random.randint(100)],
            ['17-10-2021', '00:03:48', 'A', 'a3', '0009', np.random.randint(100)],
            ['17-10-2021', '00:05:24', 'B', 'b1', '0010', np.random.randint(100)],
            ['17-10-2021', '00:05:36', 'B', 'b1', '0011', np.random.randint(100)],
            ['17-10-2021', '00:05:44', 'B', 'b2', '0012', np.random.randint(100)],
            ['17-10-2021', '00:06:03', 'B', 'b2', '0013', np.random.randint(100)],
            ['17-10-2021', '03:12:13', 'B', 'b3', '0014', np.random.randint(100)],
            ['17-10-2021', '03:12:46', 'C', 'c1', '0015', np.random.randint(100)],
            ['17-10-2021', '13:06:54', 'C', 'c1', '0016', np.random.randint(100)], 
            ['17-10-2021', '13:12:10', 'C', 'c2', '0017', np.random.randint(100)], 
            ['17-10-2021', '19:48:30', 'C', 'c2', '0018', np.random.randint(100)], 

            ['18-10-2021', '00:06:05', 'C', 'c1', '0019', np.random.randint(100)], 
            ['18-10-2021', '03:08:03', 'C', 'c2', '0020', np.random.randint(100)],
            ['18-10-2021', '12:07:08', 'C', 'c3', '0021', np.random.randint(100)],
    
            ['19-10-2021', '00:04:03', 'A', 'a1', '0022', np.random.randint(100)], 
            ['19-10-2021', '02:08:20', 'A', 'a2', '0023', np.random.randint(100)],
            ['19-10-2021', '10:03:08', 'B', 'a2', '0024', np.random.randint(100)],
            ['19-10-2021', '12:07:08', 'B', 'b1', '0025', np.random.randint(100)],
    
], columns=['str_date', 'str_time','group', 'sub_group','val_id', 'value'])

data['datetime'] = data.apply(lambda row:  dt.datetime.strptime(f'{row.str_date} {row.str_time}', '%d-%m-%Y %H:%M:%S'), axis=1)

data['date'] = data.datetime.dt.date
data['time'] = data.datetime.dt.time
data['week'] = data.datetime.dt.isocalendar().week # Получить номер недели

# Распределение

In [5]:
pd.DataFrame({
    'counts': data['sub_group'].value_counts(),
    'percents': np.round(data['sub_group'].value_counts(normalize=True)*100).astype('int').astype('str') + '%'
})

,counts,percents
c1,5,20%
a1,4,16%
b1,4,16%
b2,3,12%
a2,3,12%
c2,3,12%
a3,1,4%
b3,1,4%
c3,1,4%


### Бины

In [6]:
data['value'].value_counts(bins=3).sort_index()

(-0.1, 33.0]    9
(33.0, 66.0]    7
(66.0, 99.0]    9
Name: value, dtype: int64

### Кастомные бины

In [7]:
bins = [-1, 0, 20, 40, 60, 80, 100, 1000]

labels = ['0', '0-20', '20-40', '40-60', '60-80', '80-100', '100+']

# С помощью функции pd.cut мы можно легко разделить ряд значений на нужные нам бины. 
custom_value_counts = pd.cut(data['value'], bins=bins, labels=labels).value_counts()
custom_value_counts.index = labels
          
custom_value_counts

0         9
0-20      6
20-40     5
40-60     4
60-80     1
80-100    0
100+      0
Name: value, dtype: int64

### Кастомные бины для datetime

In [8]:
# Посчитаем количество секунд между записями
data['seconds_diff'] = (
    
    data['datetime']
                .diff()
                .fillna(pd.Timedelta(days=99))
)

# Когда дело касается построения временных бинов, гораздо удобнее работать с timedelta
time_bins = [
    pd.Timedelta(seconds = 0),
    pd.Timedelta(seconds = 30),
    pd.Timedelta(minutes = 1),
    pd.Timedelta(minutes = 5),
    pd.Timedelta(hours = 1),
    pd.Timedelta(hours = 2),
    pd.Timedelta(hours = 4),
    pd.Timedelta(hours = 6),
    pd.Timedelta(hours = 12),
    pd.Timedelta(hours = 18),
    pd.Timedelta(hours = 23),
    pd.Timedelta(days = 2),
    pd.Timedelta(days = 100)
]

labels = ['0-30seconds','30seconds-1minute', 
          '1-5minutes', '5minutes-1hours', 
          '1-2hours', '2-4hours', '4-6hours', '6-12hours', '12-18hours', '18-23hours', 
          '1-2 days', '2+ days']

span_value_counts = pd.cut(data['seconds_diff'], time_bins, labels = labels).value_counts().rename('Время между сообщениями')
span_value_counts.index = labels

span_value_counts

0-30seconds          9
30seconds-1minute    5
1-5minutes           4
5minutes-1hours      3
1-2hours             1
2-4hours             1
4-6hours             1
6-12hours            1
12-18hours           0
18-23hours           0
1-2 days             0
2+ days              0
Name: Время между сообщениями, dtype: int64

Распределения можно легко визуализировать [[Ссылка на ноутбуки с визуализациями]]

# Маски

In [9]:
group_mask = (data.group == 'A')
sub_group_mask = (data.sub_group == 'a1')

data[group_mask & sub_group_mask]

,str_date,str_time,group,sub_group,val_id,value,datetime,date,time,week,seconds_diff
0,16-10-2021,23:56:10,A,a1,0001,82,2021-10-16 23:56:10,2021-10-16,23:56:10,41,99 days 00:00:00
1,16-10-2021,23:58:30,A,a1,0002,32,2021-10-16 23:58:30,2021-10-16,23:58:30,41,0 days 00:02:20
6,17-10-2021,00:03:05,A,a1,0007,59,2021-10-17 00:03:05,2021-10-17,00:03:05,41,0 days 00:03:23
21,19-10-2021,00:04:03,A,a1,0022,4,2021-10-19 00:04:03,2021-10-19,00:04:03,42,0 days 11:56:55


# Векторизованные операции над строками

## Методы, аналогичные строковым методам языка Python

| Methond                      | Description                                                                                            | Documentation                |
| -----------------------------|--------------------------------------------------------------------------------------------------------|------------------------------|
| len()                        | Возвращает длинну строки                                                                               | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.len.html                                                                 |
| ljust(width, fillchar=' ')   | Добавляет к строке указанные символы справа, пока строка не достигнет указанной длинны                 | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.ljust.html                                                               |
| rjust(width, fillchar=' ')   | Добавляет к строке указанные символы слева, пока строка не достигнет указанной длинны                  | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.rjust.html#pandas.Series.str.rjust                                       |
| center(width, fillchar=' ')  | Равномерно добавляет к строке указанные символы слева и справа, пока она не достигнет указанной длинны | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.center.html
| zfill(width)                 | Добавляет к строке нули пока она не достигнет указанной длинны                                         | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.zfill.html                                                               |
| strip(to_strip=' ')          | Удаляет указанные символы и их комбинации в начале и конце строки, по умолчанию удаляются пробелы      |                                 https://pandas.pydata.org/pandas-docs/version/0.24/reference/api/pandas.Series.str.strip.html                                           |                               
| rstrip(to_strip=' ')         | Удаляет указанные символы и их комбинации в конце строки, по умолчанию удаляются пробелы               | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.rstrip.html                                                              |
| lstrip(to_strip=' ')         | Удаляет указанные символы и их комбинации в начале строки, по умолчанию удаляются пробелы              | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.lstrip.html                                                              |


In [63]:
df = pd.DataFrame({
    'String': ['1', '22', '333', '4444', '55555', '_A_']
})

df['len()'] = df.String.str.len()

df['ljust()'] = df.String.str.ljust(width=5, fillchar='_')
df['rjust()'] = df.String.str.rjust(width=5, fillchar='_')
df['center()'] = df.String.str.center(width=5, fillchar='_')
df['zfill()'] = df.String.str.zfill(width=5)

df['rstrip()'] = df.String.str.rstrip(to_strip='_')
df['lstrip()'] = df.String.str.lstrip(to_strip='_')
df['strip()'] = df.String.str.strip(to_strip='_')


df

,String,len(),ljust(),rjust(),center(),zfill(),rstrip(),lstrip(),strip()
0,1,1,1____,____1,__1__,00001,1,1,1
1,22,2,22___,___22,__22_,00022,22,22,22
2,333,3,333__,__333,_333_,00333,333,333,333
3,4444,4,4444_,_4444,_4444,04444,4444,4444,4444
4,55555,5,55555,55555,55555,55555,55555,55555,55555
5,_A_,3,_A___,___A_,__A__,00_A_,_A,A_,A


---

| Methond                  | Description                                                                                                   | Documentation                           |
| ------------------------ | --------------------------------------------------------------------------------------------------------------|-----------------------------------------|
| lower()                  |Перевести в нижний регистр                                                                                     | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.lower.html#pandas.Series.str.lower                                          |
| upper()                  |Перевести в верхний регистр                                                                                    | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.upper.html                                                                  |
| capitalize()             |Переводит первую букву в верхний регистр и оставляет остальные буквы в нижнем                                  | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.capitalize.html#pandas.Series.str.capitalize                                |
| title()                  |Переводит первую букву каждого слова в верхний регистр и оставляет остальные буквы в нижнем                    | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.title.html#pandas.Series.str.title                                          |
| swapcase()               |Переводит буквы в верхнем регистре в нижний, а в нижнем регистре - в верхний                                   | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.swapcase.html#pandas.Series.str.swapcase                                    |

In [64]:
df = pd.DataFrame({
    'Sentence': ['Это предложение начинается с большой буквы', 'здесь все группы маленькие', 'А ЗДЕСЬ ВСЕ БОЛЬШИЕ', 'Каждое Слово С Большой Буквы']
})

df['lower()'] = df.Sentence.str.lower()
df['upper()'] = df.Sentence.str.upper()
df['capitalize()'] = df.Sentence.str.capitalize()
df['title()'] = df.Sentence.str.title()
df['swapcase()'] = df.Sentence.str.swapcase()

df

,Sentence,lower(),upper(),capitalize(),title(),swapcase()
0,Это предложение начинается с большой буквы,это предложение начинается с большой буквы,ЭТО ПРЕДЛОЖЕНИЕ НАЧИНАЕТСЯ С БОЛЬШОЙ БУКВЫ,Это предложение начинается с большой буквы,Это Предложение Начинается С Большой Буквы,эТО ПРЕДЛОЖЕНИЕ НАЧИНАЕТСЯ С БОЛЬШОЙ БУКВЫ
1,здесь все группы маленькие,здесь все группы маленькие,ЗДЕСЬ ВСЕ ГРУППЫ МАЛЕНЬКИЕ,Здесь все группы маленькие,Здесь Все Группы Маленькие,ЗДЕСЬ ВСЕ ГРУППЫ МАЛЕНЬКИЕ
2,А ЗДЕСЬ ВСЕ БОЛЬШИЕ,а здесь все большие,А ЗДЕСЬ ВСЕ БОЛЬШИЕ,А здесь все большие,А Здесь Все Большие,а здесь все большие
3,Каждое Слово С Большой Буквы,каждое слово с большой буквы,КАЖДОЕ СЛОВО С БОЛЬШОЙ БУКВЫ,Каждое слово с большой буквы,Каждое Слово С Большой Буквы,кАЖДОЕ сЛОВО с бОЛЬШОЙ бУКВЫ


---

| Methond                  | Description                                                                                                  | Documentation                              |
| ------------------------ | -------------------------------------------------------------------------------------------------------------| -------------------------------------------|
| isspace()                | Проверяет все ли символы являются пробелами                                                                  | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.isspace.html                                                               | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.islower.html                                                               |
| islower()                | Проверяет все ли символы записаны в нижнем регистре                                                          |                                                         https://pandas.pydata.org/docs/reference/api/pandas.Series.str.islower.html                                                               |
| isupper()               | Проверяет все ли символы записаны в верхнем регистре                                                          | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.isupper.html                                                               |
| istitle()                | Проверяет все ли слова в строке начинаются с большой буквы. Слова написанные в верхнем регистре не считаются | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.istitle.html#pandas.Series.str.istitle |

In [65]:
df = pd.DataFrame({
    'Sentence': ['Это предложение начинается с большой буквы', 'здесь все группы маленькие', 'А ЗДЕСЬ ВСЕ БОЛЬШИЕ', 'Каждое Слово С Большой Буквы', '     ']
})

df['isspace()'] = df.Sentence.str.isspace()
df['islower()'] = df.Sentence.str.islower()
df['isupper()'] = df.Sentence.str.isupper()
df['istitle()'] = df.Sentence.str.istitle()

df

,Sentence,isspace(),islower(),isupper(),istitle()
0,Это предложение начинается с большой буквы,False,False,False,False
1,здесь все группы маленькие,False,True,False,False
2,А ЗДЕСЬ ВСЕ БОЛЬШИЕ,False,False,True,False
3,Каждое Слово С Большой Буквы,False,False,False,True
4,,True,False,False,False


---

| Methond                  | Description                                                                                                   | Documentation                  |
| ------------------------ | --------------------------------------------------------------------------------------------------------------|--------------------------------|
| find()                   |Возвращает наименьший индекс указанной подстроки, или -1 если подстрока не найдена                             | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.find.html                                                                   |
| rfind()                  |Возвращает наибольший индекс указанной подстроки, или -1 если подстрока не найдена                             | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.rfind.html                                                                  |
| index()                  |Работает также как и find(), но если подстрока не найдена выводит ValueError                                   | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.index.html                                                                  |
| rindex()                 |Работает также как и rfind(), но если подстрока не найдена выводит ValueError                                  | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.rindex.html                                                                 |

In [68]:
df = pd.DataFrame({'Sentence': ['Здесь всего одни key', 'key вначале, key в конце', 'Здесь ничего нет']})

df['find()'] = df.Sentence.str.find('key')
df['rfind()'] = df.Sentence.str.rfind('key')

df

,Sentence,find(),rfind()
0,Здесь всего одни key,17,17
1,"key вначале, key в конце",0,13
2,Здесь ничего нет,-1,-1


---

| Methond                          | Description                                                                                                                            | Documentation            |
| ---------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|--------------------------|
| translate(table={})              |Переводит символы в строке по указанному словарю                                                                                        | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.translate.html                                                                                               |
| replace(pat, repr, ...)          | Заменеят каждое вхождение шаблона указанным значением                                                                                  | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.rfind.html                                                                                                   |
| startswith(pat='', na=None)      |Проверяет начинается ли строка с указанного патерна, возвращает булево значение и элемент na для значений, которые не являются строками | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.startswith.html                                                                                              |
| endswith(pat='', na=None)        |Проверяет начинается ли строка с указанного патерна, возвращает булево значение и элемент na для значений, которые не являются строками | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.endswith.html                                                                                                |

---

| Methond                  | Description                                                                  | Documentation                      |
| ------------------------ | -----------------------------------------------------------------------------| -----------------------------------|
| isalnum()                | Проверяет, все ли символы в каждой строке являются буквенно-цифровыми        | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.isalnum.html#pandas.Series.str.isalnum     |
| isalpha()                | Проверяет, все ли символы являются буквенными                                | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.isalpha.html#pandas.Series.str.isalpha     |
| isdigit()                | Проверяет, все ли символы являются цифрами.                                  | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.isdigit.html#pandas.Series.str.isdigit     |
| isnumeric()              | Проверяет, все ли символы являются числовыми                                 | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.isnumeric.html#pandas.Series.str.isnumeric |
| isdecimal()              | Проверяет, все ли символы являются десятичными                               | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.isdecimal.html#pandas.Series.str.isdecimal |


In [72]:
df = pd.DataFrame({
    'Sentence': ['Два и 5', 'Только буквы', 'letters only', 'Two and 5', '', '+7903383383', '8903383383', '8903383383', 'one', 'two']
})

df['isalnum()'] = df.Sentence.str.isalnum()
df['isalpha()'] = df.Sentence.str.isalpha()
df['isdigit()'] = df.Sentence.str.isdigit()
df['isnumeric()'] = df.Sentence.str.isnumeric()
df['isdecimal()'] = df.Sentence.str.isdecimal()

df

,Sentence,isalnum(),isalpha(),isdigit(),isnumeric(),isdecimal()
0,Два и 5,False,False,False,False,False
1,Только буквы,False,False,False,False,False
2,letters only,False,False,False,False,False
3,Two and 5,False,False,False,False,False
4,,False,False,False,False,False
5,+7903383383,False,False,False,False,False
6,8903383383,True,False,True,True,True
7,8903383383,True,False,True,True,True
8,one,True,True,False,False,False
9,two,True,True,False,False,False


---

| Methond                        | Description                                              | Documentation |
| -------------------------------| ---------------------------------------------------------| --------------|
| split(pat=' ', ...)            | Разделить строку по указанной подстроке, начиная с начала| https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html
| rsplit(pat=' ', ...)           | Разделить строку по указанной подстроке, нечиная с конца | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.rsplit.html#pandas.Series.str.rsplit
| join(sep=' ')                  | Объединяет списки по указанному разделителю              | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.join.html#pandas.Series.str.join
| partition(sep=' ', expand=True)| Разделяет строку по первому появлению указанной подстроки, начиная с начала, всегда возвращает три элемента| https://pandas.pydata.org/docs/reference/api/pandas.Series.str.partition.html |
| rpartition(sep=' ', expand=True)| Разделяет строку по первому появлению указанной подстроки, начиная с конца, всегда возвращает три элемента| https://pandas.pydata.org/docs/reference/api/pandas.Series.str.rpartition.html#pandas.Series.str.rpartition |

---

| Methond                         | Description                                                                                    | Documentation                                                             |
| ------------------------------- | ---------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------- |
| get(i=int)                      |  Индексирует все элементы                                                                      | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get.html   |
| slice(start, stop, step)        |  Вызывает подстроку из каждого элемента                                                        | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html |
| slice_replace(start, stop, repl)|  Земеняет в каждом элементе вырезанную подстроку заданным значением                            | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice_replace.html |
| cat(others=[...], sep=' ')      |  Конкатенация строк                                                                            | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.cat.html |
| repeat(repeats=int)             |  Повторяет значения (указанное число раз)                                                      | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.repeat.html |
| normalize(form=' ')             |  Возвращает версию строки в кодировке Unicode                                                  | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.normalize.html |
| pad(width, side, fillchar)      |  Добавляет пробелы слева, справа или с обеих сторон строки                                     | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html |
| wrap(width, ...)                |  Разбивает длинные строковые значения на строки длины, не превышающей заданную                 | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.wrap.html |
| join(sep=' ')                   |  Объединяйте списки, содержащиеся в виде элементов в серии/индексе, с переданным разделителем  | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.join.html |
| get_dummies(sep=' ')            |  Извлекает значения переменных-идентификаторов в виде объекта DataFrame                        | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get_dummies.html |

## Функция pd.get_dummies()
- https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [76]:
df = pd.DataFrame({
    'A': ['a', 'b', 'a'], 
    'B': ['b', 'a', 'c'],
    'C': [1, 2, 3]
})

df

,A,B,C
0,a,b,1
1,b,a,2
2,a,c,3


In [77]:
pd.get_dummies(df, prefix=['col1', 'col2'])

,C,col1_a,col1_b,col2_a,col2_b,col2_c
0,1,1,0,0,1,0
1,2,0,1,1,0,0
2,3,1,0,0,0,1


## ReGex и методы использующие регулярные выражения

### ReGex

Полезные ресурсы:
- https://habr.com/ru/post/349860/



| Symbol     | Description                                                            |
| ---------- | ---------------------------------------------------------------------- |
| .          | Задает **один** произвольный символ                                    |
| **[]**     | Заменяет символ из квадратных скобок                                   |
| -          | Задает символ, которого не должно быть в скобках                       |
| **[^]**    | Задает один символ из **не** содержащихся в квадратных скобках         |
| ^          | Обозначает начало последовательности                                   |
| ?          | Обозначает строго одно повторение символа                              |
| +          | Обозначает один символ, который повторяется несколько раз              |
| |          | Логическое **ИЛИ**. Либо выражение до, либо выражение после символа    |
| \          | Экранирование. Для использования метасимволов в качестве обычных       |
| ()         | Группирует символы внутри                                              |
| {}         | Указывается число повторений предыдущего символа                       |

| Construction | Description                                                          |
| ------------ | -------------------------------------------------------------------- |
| \b           | Начало или конец слова (слева пусто или не-буква, справа буква и наоборот). Соответствует позиции а не символу |
| \B           | Не граница слова: либо и слева, и справа буквы, либо и слева, и справа **не** буквы                            |
| \d           | Соответствует любой одной цифре и заменяет собой выражение [0-9]     |
| \D           | Исключает все цифры и заменяет [^0-9]                                |
| \w           | Заменяет любую цифру, букву, а также знак ниженего подчеркивания     |
| \W           | Любой символ кроме латиницы, цифр или нижнего подчеркивания          |
| \s           | Соответствует любому пробельному символу                             |
| \S           | Описывают любой **не**пробельный символ                              |
| [..]         | Один из символов в скобках, а также любой символ из диапазона a-b    |
| [^..]        | Любой символ кроме перечисленных                                     |

### Полезные паттерны

### Методы python

| Methond                  | Description                                                                                 | Documentation                                                               |
| ------------------------ | ------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------- |
| match(pat, ...)          |  Вызывает функцию re.match() для каждого элемента, возвращая булево значение                | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.match.html   |
| extract(pat, ...)        |  Вызывает функцию re.match() для каждого элемента, возвращая подходящие группы в виде строк | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extract.html |
| findall(pat, ...)        |  Вызвает функцию re.findall() для каждого элемента                                          | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.findall.html |
| replace(pat, ...)        |  Заменяет вхождения шаблона какой-нибудь другой строкой                                     | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html |
| contains(pat, ...)       |  Вызывает функцию re.search() для каждого элемента, возвращая булево значение               | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html|
| count(pat, ...)          |  Подсчитывает вхождения шаблона                                                             | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.count.html   |
| split(pat, ...)          |  Эквивалент функции str.split(), но принимающий на входе регулярные выражения               | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html   |
| rsplit(pat, ...)         |  Эквивалент функции str.rsplit(), но принимающий на входе регулярные выражения              | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.rsplit.html  |

# Полезные практики

## Объединить несколько строковых колонок в одну

In [42]:
df = pd.DataFrame({
    'col1': ['1', 'a', 'alpha'],
    'col2': ['2', 'b', 'betta'],
    'col3': ['3', 'c', 'gamma']
})

df['joint_cols_v1'] = df['col1'] + "_" + df['col2'] + "_" + df['col3']
df['joint_cols_v2'] = df['col1'].str.cat([df['col2'], df['col3']], sep=' - ')
df

,col1,col2,col3,joint_cols_v1,joint_cols_v2
0,1,2,3,1_2_3,1 - 2 - 3
1,a,b,c,a_b_c,a - b - c
2,alpha,betta,gamma,alpha_betta_gamma,alpha - betta - gamma


## Разделить одну строковую колонку на несколько

In [55]:
df = pd.DataFrame({'united_col': ['1_2_3', 'a_b_c', 'alpha_betta_gamma']})
df[['col1', 'col2', 'col3']] = df['united_col'].str.split('_').apply(pd.Series)
df

,united_col,col1,col2,col3
0,1_2_3,1,2,3
1,a_b_c,a,b,c
2,alpha_betta_gamma,alpha,betta,gamma


### Объединить списки в колонке в строку

In [43]:
df = pd.DataFrame({'lists': [['a', 'b', 'c'], [1, 2, 3], ['alpha', 'betta', 'gamma']]})
df['joint_lists'] = df.lists.str.join('_')
df['joint_lists_with_numbers'] = (
    df.lists
      .apply(lambda l: list(map(str, l)) if any([not isinstance(el, str) for el in l]) else l)
      .str.join('_')
)
df

,lists,joint_lists,joint_lists_with_numbers
0,"[a, b, c]",a_b_c,a_b_c
1,"[1, 2, 3]",NaN,1_2_3
2,"[alpha, betta, gamma]",alpha_betta_gamma,alpha_betta_gamma


### Посчитать количество слов в предложениии

In [73]:
df = pd.DataFrame(['Одно', 'Два слова', 'Здесь три слова', 'Здесь всего четыре слова', 'Здесь как бы пять слов'], columns=['Sentence'])
df['number_of_words'] = df.Sentence.str.split().str.len()
df

,Sentence,number_of_words
0,Одно,1
1,Два слова,2
2,Здесь три слова,3
3,Здесь всего четыре слова,4
4,Здесь как бы пять слов,5
